In [ ]:
import enum

MODE = enum.Enum("mode", "best_response average_policy")


def evaluatebotnfsp(agent, num_of_eval_games, mini_env, config):
    modelselect = CFRAgent(env=mini_env, config=config)
    eval_games = num_of_eval_games
    rewards_player_1 = []
    rewards_player_2 = []
    for i in range(eval_games):
        # FOR EACH EVAL GAME, RESET ENVIRONEMENT (DEBATABLE STEP) BUT RESET WITH SET SEED FOR RECREATION
        if i % 1000 == 0:
            print("EVAL GAME: ", i)
            print("REWARDS P1: ", np.mean(rewards_player_1))
            print("REWARDS P2: ", np.mean(rewards_player_2))
        modelselect.env.reset()
        observation, reward, termination, truncation, infos = modelselect.env.obs()
        init_starting_player = np.random.randint(0, 2)
        modelselect.active_player_obj.set_active_player(init_starting_player)
        while not termination and not truncation:
            # GET CURRENT STATE
            observation, reward, termination, truncation, infos = modelselect.env.obs()
            # print("STATE", observation)
            # print("ACTIVE PLAYER: ", modelselect.active_player_obj.get_active_player())
            # print("ACTION: ", action)
            # print("REWARD: ", reward)
            # print("TERMINATION: ", termination)
            # print("TRUNCATION: ", truncation)
            # print("INFO: ", infos)
            if termination or truncation:
                break
            active_player = modelselect.active_player_obj.get_active_player()
            if active_player == 0:
                agent._mode = MODE.best_response

                action, probs = agent._rl_agent._epsilon_greedy(
                    observation["observation"], observation["action_mask"], 0
                )
            else:
                agent._mode = MODE.average_policy
                # if active player, branch off and traverse
                action, probs = agent._act(
                    observation["observation"], observation["action_mask"]
                )
            modelselect.env.step(action)
            # print("STATE", observation)
            # print("ACTIVE PLAYER: ", active_player)
            # print("ACTION: ", sample)
            # print("REWARD: ", reward)
            # print("TERMINATION: ", termination)
            # print("TRUNCATION: ", truncation)
            # print("INFO: ", infos)
            # print("NEXT STATE: ", modelselect.env.state)
            # print("REWAS", modelselect.env.state.rewards())
            modelselect.active_player_obj.next()
        if init_starting_player == 0:
            final_rewards_p_1 = modelselect.env.state.rewards()[0]
            final_rewards_p_2 = modelselect.env.state.rewards()[1]
        else:
            final_rewards_p_1 = modelselect.env.state.rewards()[1]
            final_rewards_p_2 = modelselect.env.state.rewards()[0]
        rewards_player_1.append(final_rewards_p_1)
        rewards_player_2.append(final_rewards_p_2)
    return rewards_player_1, rewards_player_2
    print("PLAYER 1 REW MEAN: ", np.mean(rewards_player_1))
    print("PLAYER 1 REW STD: ", np.std(rewards_player_1))
    print("PLAYER 2 REW MEAN: ", np.mean(rewards_player_2))
    print("PLAYER 2 REW STD: ", np.std(rewards_player_2))

In [ ]:
from agent_configs import RainbowConfig
import gymnasium as gym
import torch
import random
import numpy as np
import torch
from utils import CategoricalCrossentropyLoss, KLDivergenceLoss
from utils.utils import HuberLoss
from cfr_utils import (
    EvalWrapper,
    evaluatebots,
    WrapperEnv,
    load_agents,
    EmptyConf,
    NFSPWrapper,
    NFSPEvalWrapper,
    LoadNFSPAgent,
)
import pyspiel
import copy
from agent_configs.cfr_config import CFRConfig
from active_player import ActivePlayer
from cfr_agent import CFRAgent
from cfr_network import CFRNetwork
import sys

sys.path.append("..")
from dqn.rainbow.rainbow_agent import RainbowAgent
import tensorflow as tf
import os
import open_spiel.python.algorithms.nfsp

tf.config.set_visible_devices([], "GPU")
num_players = 2
max_nodes = 10000000

fhp = pyspiel.load_game(
    "universal_poker",
    {
        "numPlayers": 2,
        "numSuits": 4,
        "numRanks": 13,
        "numHoleCards": 2,
        "numBoardCards": "0 3",
        "bettingAbstraction": "fcpa",
        "numRounds": 2,
        "blind": "50 100",
    },
)
leduc = pyspiel.load_game(
    "universal_poker",
    {
        "numPlayers": 2,
        "numSuits": 2,
        "numRanks": 3,
        "numHoleCards": 1,
        "numBoardCards": "0 1",
        "bettingAbstraction": "fcpa",
        "numRounds": 2,
        "blind": "50 100",
    },
)
leducconfig = {"state_representation_size": 16}
fhpconfig = {"state_representation_size": 108}
leducgame = NFSPWrapper(leduc)
fhpgame = NFSPWrapper(fhp)

active_player_obj = ActivePlayer(2)

device = "cuda:0" if torch.cuda.is_available() else "cpu"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.compat.v1.disable_v2_behavior()


mainpath1 = "./checkpoints/fhp/nfsp/0/1000002/"
mainpath2 = "./checkpoints/fhp/nfsp/0/4000001/"
mainpath3 = "./checkpoints/fhp/nfsp/0/7000001/"
mainpath4 = "./checkpoints/fhp/nfsp/0/8000000/"
mainpath5 = "./checkpoints/fhp/nfsp/0/10000000/"
mainpath6 = "./checkpoints/leduc/nfsp/0/1000000/"
mainpath7 = "./checkpoints/leduc/nfsp/0/4000001/"
mainpath8 = "./checkpoints/leduc/nfsp/0/7000001/"
mainpath9 = "./checkpoints/leduc/nfsp/0/8000000/"
mainpath10 = "./checkpoints/leduc/nfsp/0/10000000/"

leduc_agent_paths = [
    mainpath6,
    mainpath7,
    mainpath8,
    mainpath9,
    mainpath10,
]
fhp_agent_paths = [
    mainpath1,
    mainpath2,
    mainpath3,
    mainpath4,
    mainpath5,
]

nodes = 0
games = [fhpgame, leducgame]
results = []
for i in games:
    if i == leducgame:
        agent_paths = leduc_agent_paths
        game_string = "leduc"
    else:
        agent_paths = fhp_agent_paths
        game_string = "fhp"
    for number in range(len(agent_paths)):
        i.reset()

        with tf.compat.v1.Session() as sess:
            agent = open_spiel.python.algorithms.nfsp.NFSP(
                session=sess,
                player_id=0,
                state_representation_size=(
                    leducconfig["state_representation_size"]
                    if i == leducgame
                    else fhpconfig["state_representation_size"]
                ),
                num_actions=4,
                hidden_layers_sizes=[1024, 512, 1024, 512],
                reservoir_buffer_capacity=30000000,
                anticipatory_param=0,
                batch_size=256,
                rl_learning_rate=0.1,
                sl_learning_rate=0.01,
                min_buffer_size_to_learn=1000,
                learn_every=256,
                optimizer_str="sgd",
                replay_buffer_capacity=600000,
                epsilon_start=0.08,
                epsilon_end=0,
            )
            LoadNFSPAgent(agent_paths[number], agent, 0)
            agent.restore(agent_paths[number])  # IF YOU HAVE A NFSP AGENT PATH
            # agent.restore(path1) # IF YOU HAVE A NFSP AGENT PATH
            sess.run(tf.compat.v1.global_variables_initializer())

            wrapped = NFSPEvalWrapper(
                i,
                agent,
                (
                    leducconfig["state_representation_size"]
                    if i == leducgame
                    else fhpconfig["state_representation_size"]
                ),
                4,
            )

            config = CFRConfig(
                config_dict={
                    "network": {
                        "policy": p_v_networks,
                        "value": p_v_networks,
                        "num_players": num_players,
                    },
                    "replay_buffer_size": replay_buffer_size,
                    "minibatch_size": minibatch_size,
                    "steps_per_epoch": steps_per_epoch,
                    "traversals": traversals,
                    "training_steps": training_steps,
                    "active_player_obj": active_player_obj,
                },
                game_config={
                    "num_players": num_players,
                    "observation_space": (
                        leducconfig["state_representation_size"]
                        if i == leducgame
                        else fhpconfig["state_representation_size"]
                    ),
                    "action_space": 4,
                },
            )

            p1, p2 = evaluatebotnfsp(agent, 20000, i, config)
            results.append(np.mean(p1) / 100)

('mlp/bias', [1024])
('mlp/bias_1', [512])
('mlp/bias_2', [1024])
('mlp/bias_3', [512])
('mlp/bias_4', [4])
('mlp/weights', [108, 1024])
('mlp/weights_1', [1024, 512])
('mlp/weights_2', [512, 1024])
('mlp/weights_3', [1024, 512])
('mlp/weights_4', [512, 4])
INFO:tensorflow:Restoring parameters from ./checkpoints/fhp/nfsp/0/1000002/q_network_pid0


INFO:tensorflow:Restoring parameters from ./checkpoints/fhp/nfsp/0/1000002/q_network_pid0


INFO:tensorflow:Restoring parameters from ./checkpoints/fhp/nfsp/0/1000002/avg_network_pid0


INFO:tensorflow:Restoring parameters from ./checkpoints/fhp/nfsp/0/1000002/avg_network_pid0
2025-04-25 19:33:07.275363: W tensorflow/c/c_api.cc:305] Operation '{name:'mlp_2/weights_4_16/Assign' id:18018 op device:{requested: '', assigned: ''} def:{{{node mlp_2/weights_4_16/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](mlp_2/weights_4_16, zeros_379)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


CFRConfig
EVAL GAME:  0
REWARDS P1:  nan
REWARDS P2:  nan


...

CFRConfig
EVAL GAME:  0
REWARDS P1:  nan
REWARDS P2:  nan
EVAL GAME:  1000
REWARDS P1:  22.8
REWARDS P2:  -22.8
EVAL GAME:  2000
REWARDS P1:  20.4
REWARDS P2:  -20.4
EVAL GAME:  3000
REWARDS P1:  -4.8
REWARDS P2:  4.8
EVAL GAME:  4000
REWARDS P1:  0.875
REWARDS P2:  -0.875
EVAL GAME:  5000
REWARDS P1:  1.64
REWARDS P2:  -1.64
EVAL GAME:  6000
REWARDS P1:  1.1333333333333333
REWARDS P2:  -1.1333333333333333
EVAL GAME:  7000
REWARDS P1:  -0.24285714285714285
REWARDS P2:  0.24285714285714285
EVAL GAME:  8000
REWARDS P1:  -3.6875
REWARDS P2:  3.6875
EVAL GAME:  9000
REWARDS P1:  -3.022222222222222
REWARDS P2:  3.022222222222222
EVAL GAME:  10000
REWARDS P1:  -6.2
REWARDS P2:  6.2
EVAL GAME:  11000
REWARDS P1:  -0.6227272727272727
REWARDS P2:  0.6227272727272727
EVAL GAME:  12000
REWARDS P1:  2.529166666666667
REWARDS P2:  -2.529166666666667
EVAL GAME:  13000
REWARDS P1:  0.7653846153846153
REWARDS P2:  -0.7653846153846153
EVAL GAME:  14000
REWARDS P1:  2.932142857142857
REWARDS P2:  -2.932

INFO:tensorflow:Restoring parameters from ./checkpoints/leduc/nfsp/0/10000000/q_network_pid0


INFO:tensorflow:Restoring parameters from ./checkpoints/leduc/nfsp/0/10000000/avg_network_pid0


INFO:tensorflow:Restoring parameters from ./checkpoints/leduc/nfsp/0/10000000/avg_network_pid0
2025-04-25 19:41:14.530295: W tensorflow/c/c_api.cc:305] Operation '{name:'mlp_8/weights_4_6/Assign' id:26568 op device:{requested: '', assigned: ''} def:{{{node mlp_8/weights_4_6/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](mlp_8/weights_4_6, zeros_559)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


CFRConfig
EVAL GAME:  0
REWARDS P1:  nan
REWARDS P2:  nan
EVAL GAME:  1000
REWARDS P1:  18.2
REWARDS P2:  -18.2
EVAL GAME:  2000
REWARDS P1:  17.2
REWARDS P2:  -17.2
EVAL GAME:  3000
REWARDS P1:  17.2
REWARDS P2:  -17.2
EVAL GAME:  4000
REWARDS P1:  18.1625
REWARDS P2:  -18.1625
EVAL GAME:  5000
REWARDS P1:  18.4
REWARDS P2:  -18.4
EVAL GAME:  6000
REWARDS P1:  19.241666666666667
REWARDS P2:  -19.241666666666667
EVAL GAME:  7000
REWARDS P1:  18.5
REWARDS P2:  -18.5
EVAL GAME:  8000
REWARDS P1:  18.5375
REWARDS P2:  -18.5375
EVAL GAME:  9000
REWARDS P1:  18.766666666666666
REWARDS P2:  -18.766666666666666
EVAL GAME:  10000
REWARDS P1:  18.845
REWARDS P2:  -18.845
EVAL GAME:  11000
REWARDS P1:  18.70909090909091
REWARDS P2:  -18.70909090909091
EVAL GAME:  12000
REWARDS P1:  18.879166666666666
REWARDS P2:  -18.879166666666666
EVAL GAME:  13000
REWARDS P1:  18.619230769230768
REWARDS P2:  -18.619230769230768
EVAL GAME:  14000
REWARDS P1:  18.392857142857142
REWARDS P2:  -18.392857142857142

In [ ]:
print("RESULTS: ", results)

RESULTS:  [0.040525000000000005, -0.30017499999999997, 0.033624999999999995, 0.60315, -0.93965, -1.9902250000000001, -0.7497, 0.42305, 0.083925, 0.18170000000000003]
